## Cutting GRID 
Memotong raster dengan sampel besar menjadi beberapa raster yang lebih kecil

#### [00] Load Library

In [ ]:
import pandas as pd
import numpy as np
import math

import geopandas as gpd 
import rasterio 
from shapely.geometry import Polygon
from osgeo import gdal

from google.cloud import storage
import gcsfs

from datetime import datetime
import glob
import os
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

from omegaconf import DictConfig,OmegaConf




**!! Edit kdPIC parameter in config/config.yaml**

specify the "kdPIC" and the band that would be calculated

In [ ]:
conf = OmegaConf.load('config/config.yaml')

In [ ]:
kdPIC=conf['config']['pic_']

#### [01] Import Data 

In [ ]:
file_sampel = kdPIC + "_sample2022.gpkg"
sampel = gpd.read_file("gs://bps-gcp-bucket/MLST2023/sample/"+file_sampel).to_crs(4326)
grid = gpd.read_file("gs://bps-gcp-bucket/MLST2023/sample/Grid_Ekoreg_2022.gpkg").query('kdPIC=='+'\"'+kdPIC+'\"')

#### [02] Function Definition


**[02.01]** Mendapat nama file raster

In [ ]:
def get_name(grid, pic):
    filename= 'duatahun_'+grid+'_QALPN1_PakKus_sentinel2_10m.tif'
    filename_full = '/vsigs/bps-gcp-bucket/citra-sentinel2/'+pic+'/'+filename
    return filename, filename_full


**[02.02]** Mendapat id grid terhadap treshold tertentu

In [ ]:
def get_id(sampel, treshold):
    id_ = pd.pivot_table(sampel, index="ID_GRID", aggfunc="count").sort_values(by="geometry",ascending=False).query("geometry>"+str(treshold)).index
    return list(id_)

**[02.03]** Memotong raster grid

In [ ]:
def create_grid_raster(id_grid, n): 
    filename, filename_full = get_name(id_grid, kdPIC)
    
    ds = gdal.Open(filename_full)
    band = ds.GetRasterBand(1)
    xsize = band.XSize
    ysize = band.YSize

    tile_size_x = math.ceil(xsize/n)
    tile_size_y = math.ceil(ysize/n)

    k = 1
    for i in range(0, xsize, tile_size_x):
        for j in range(0, ysize, tile_size_y):
            ds = gdal.Translate('ml_output/00_cutting_to_subgrid/' + filename[0:9] +  id_grid + "_"  + str(k) + filename[-32:], filename_full, srcWin = [i, j, tile_size_x, tile_size_y])
            ds = None
            k+=1
            

**[02.04]** Memotong vector grid

In [ ]:
def create_grid_polygon(id_grid, n): 
    
    tmp = grid.loc[grid.ID_GRID == id_grid]
    xmin,ymin,xmax,ymax =  tmp.total_bounds
    width = (xmax-xmin)/n
    height = (ymax-ymin)/n
    rows = n
    cols = n
    XleftOrigin = xmin
    XrightOrigin = xmin + width
    YtopOrigin = ymax
    YbottomOrigin = ymax- height
    polygons = []

    for i in range(n):
        Ytop = YtopOrigin
        Ybottom =YbottomOrigin
        for j in range(n):
            polygons.append(Polygon([(XleftOrigin, Ytop), (XrightOrigin, Ytop), (XrightOrigin, Ybottom), (XleftOrigin, Ybottom)])) 
            Ytop = Ytop - height
            Ybottom = Ybottom - height
        XleftOrigin = XleftOrigin + width
        XrightOrigin = XrightOrigin + width
    
    sub_grid = gpd.GeoDataFrame({'id_sub':range(1,n**2+1),
                             'geometry':polygons})
    sub_grid['sub_grid'] = [id_grid + '_' + str(x) for x in sub_grid.id_sub]
    
    return sub_grid

#### [03] List of GRID dengan sampel besar

List grid dengan sampel lebih dari 1000 titik sampel

In [ ]:
id_grid_ = list(sampel.ID_GRID.unique())
id_ = get_id(sampel, 1000) ## edit dengan maksimum sampel
id_

In [ ]:
print("Total Seluruh Grid : " + str(len(sampel.ID_GRID.unique())))
print("Total Seluruh Grid dg dengan Sampel > 1000 : " + str(len(id_)))
print("Persentase Seluruh Grid dg dengan Sampel > 1000 : " + str(100*len(id_)/len(sampel.ID_GRID.unique())))


#### [04] Memotong Grid dalam [nxn] Raster

Memotong grid menjadi 25 sub raster

In [ ]:
cols_ = list(sampel.columns)
cols_.append('sub_grid')
res = pd.DataFrame(columns=cols_)

for i in tqdm(id_grid_):
    sj = sampel.loc[sampel.ID_GRID == i]
    
    if i in id_:
        create_grid_raster(i, 5) 
        sub_grid = create_grid_polygon(i, 5)
        sj_ = gpd.sjoin(sj, sub_grid[['geometry','sub_grid']])
        res = res.append(sj_)
    else:
        sj['sub_grid'] = i
        res = res.append(sj)


#### [05] Cek Jumlah Sampel tiap GRID dan SubGrid 

In [ ]:
res.groupby(['ID_GRID','sub_grid']).sub_grid.count()

##### [06] Upload File Raster dan Sampel ke Bucket dalam GCloud Storange

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'config/gcp_store.json'
client = storage.Client()
bucket = client.get_bucket('bps-gcp-bucket')

for i in glob.glob("ml_output/00_cutting_to_subgrid/*.tif"):
    tif_name = i.split('/')[2]
    gcp_file_name='citra-sentinel2/'+kdPIC+'/'+tif_name
    print(gcp_file_name)
    bucket.blob(gcp_file_name).upload_from_filename(i)
    # print(tif_name + " DONE !")
print("DONE !! SELURUH RASTER SUB GRID TELAH TERUPLOAD !")
    
gpd_file_name = "ml_output/00_cutting_to_subgrid/"+file_sampel[:-5]+"_edit.gpkg"
gpd.GeoDataFrame(res, crs="EPSG:4326", geometry="geometry").to_file(gpd_file_name,driver="GPKG")
gcp_file_name=gpd_file_name
bucket.blob("MLST2023/sample/"+kdPIC+"_sample2022_edit.gpkg").upload_from_filename(gcp_file_name)
print("DONE !! FILE GPKG EDIT TELAH TERUPLOAD !")

for i in glob.glob("ml_output/00_cutting_to_subgrid/*.tif"):
    os.remove(i)
    
for i in glob.glob("ml_output/00_cutting_to_subgrid/*.gpkg"):
    os.remove(i)    
